# FCN - Fully Convolutional Networks

In [1]:
import tensorflow as tf
import keras.backend as K
from keras.models import Sequential, Model
from keras.layers import *
from keras.applications import *

//anaconda/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
def resize_images_bilinear(X, height_factor=1, width_factor=1, target_height=None, target_width=None, data_format='default'):
    '''Resizes the images contained in a 4D tensor of shape
    - [batch, channels, height, width] (for 'channels_first' data_format)
    - [batch, height, width, channels] (for 'channels_last' data_format)
    by a factor of (height_factor, width_factor). Both factors should be
    positive integers.
    '''
    if data_format == 'default':
        data_format = K.image_data_format()
    if data_format == 'channels_first':
        original_shape = K.int_shape(X)
        if target_height and target_width:
            new_shape = tf.constant(np.array((target_height, target_width)).astype('int32'))
        else:
            new_shape = tf.shape(X)[2:]
            new_shape *= tf.constant(np.array([height_factor, width_factor]).astype('int32'))
        X = permute_dimensions(X, [0, 2, 3, 1])
        X = tf.image.resize_bilinear(X, new_shape)
        X = permute_dimensions(X, [0, 3, 1, 2])
        if target_height and target_width:
            X.set_shape((None, None, target_height, target_width))
        else:
            X.set_shape((None, None, original_shape[2] * height_factor, original_shape[3] * width_factor))
        return X
    elif data_format == 'channels_last':
        original_shape = K.int_shape(X)
        if target_height and target_width:
            new_shape = tf.constant(np.array((target_height, target_width)).astype('int32'))
        else:
            new_shape = tf.shape(X)[1:3]
            new_shape *= tf.constant(np.array([height_factor, width_factor]).astype('int32'))
        X = tf.image.resize_bilinear(X, new_shape)
        if target_height and target_width:
            X.set_shape((None, target_height, target_width, None))
        else:
            X.set_shape((None, original_shape[1] * height_factor, original_shape[2] * width_factor, None))
        return X
    else:
        raise Exception('Invalid data_format: ' + data_format)

class BilinearUpSampling2D(Layer):
    def __init__(self, size=(1, 1), target_size=None, data_format='default', **kwargs):
        if data_format == 'default':
            data_format = K.image_data_format()
        self.size = tuple(size)
        if target_size is not None:
            self.target_size = tuple(target_size)
        else:
            self.target_size = None
        assert data_format in {'channels_last', 'channels_first'}, 'data_format must be in {tf, th}'
        self.data_format = data_format
        self.input_spec = [InputSpec(ndim=4)]
        super(BilinearUpSampling2D, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_first':
            width = int(self.size[0] * input_shape[2] if input_shape[2] is not None else None)
            height = int(self.size[1] * input_shape[3] if input_shape[3] is not None else None)
            if self.target_size is not None:
                width = self.target_size[0]
                height = self.target_size[1]
            return (input_shape[0],
                    input_shape[1],
                    width,
                    height)
        elif self.data_format == 'channels_last':
            width = int(self.size[0] * input_shape[1] if input_shape[1] is not None else None)
            height = int(self.size[1] * input_shape[2] if input_shape[2] is not None else None)
            if self.target_size is not None:
                width = self.target_size[0]
                height = self.target_size[1]
            return (input_shape[0],
                    width,
                    height,
                    input_shape[3])
        else:
            raise Exception('Invalid data_format: ' + self.data_format)

    def call(self, x, mask=None):
        if self.target_size is not None:
            return resize_images_bilinear(x, target_height=self.target_size[0], target_width=self.target_size[1], data_format=self.data_format)
        else:
            return resize_images_bilinear(x, height_factor=self.size[0], width_factor=self.size[1], data_format=self.data_format)

    def get_config(self):
        config = {'size': self.size, 'target_size': self.target_size}
        base_config = super(BilinearUpSampling2D, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [3]:
def conv_pool_block(x, n_filters):
    
    conv1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(x)
    conv2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv1)
    output = MaxPooling2D(pool_size=2)(conv2)
    
    return output

def FCN_32(input_size, n_classes):
        
    inputs = Input(shape=input_size)
    pool_1 = conv_pool_block(inputs, n_filters=32)
    pool_2 = conv_pool_block(pool_1, n_filters=64)
    pool_3 = conv_pool_block(pool_2, n_filters=128)
    pool_4 = conv_pool_block(pool_3, n_filters=256)
    pool_5 = conv_pool_block(pool_4, n_filters=512)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool_5)
    conv7 = Conv2D(n_classes, (1, 1), activation='relu', padding='valid')(conv6)
    classify = BilinearUpSampling2D(size=(32, 32))(conv7)
    
    model = Model(inputs, classify)
    
    return model

def FCN_16(input_size, n_classes):
    
    inputs = Input(shape=input_size)
    pool_1 = conv_pool_block(inputs, n_filters=32)
    pool_2 = conv_pool_block(pool_1, n_filters=64)
    pool_3 = conv_pool_block(pool_2, n_filters=128)
    pool_4 = conv_pool_block(pool_3, n_filters=256)
    pool_5 = conv_pool_block(pool_4, n_filters=512)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool_5)
    conv7 = Conv2D(n_classes, (1, 1), activation='relu', padding='valid')(conv6)
    conv7_up = UpSampling2D(size=(2,2))(conv7)
    
    merged = Concatenate([pool_4, conv7_up])
    classify = BilinearUpSampling2D(size=(16, 16))(merged)
    
    model = Model(inputs, classify)
    
    return model

def FCN_8(input_size, n_classes):
    
    inputs = Input(shape=input_size)
    pool_1 = conv_pool_block(inputs, n_filters=32)
    pool_2 = conv_pool_block(pool_1, n_filters=64)
    pool_3 = conv_pool_block(pool_2, n_filters=128)
    pool_4 = conv_pool_block(pool_3, n_filters=256)
    pool_5 = conv_pool_block(pool_4, n_filters=512)
    conv6 = Conv2D(1024, (3, 3), activation='relu', padding='same')(pool_5)
    conv7 = Conv2D(n_classes, (1, 1), activation='relu', padding='valid')(conv6)
    
    conv7_up = UpSampling2D(size=(4,4))(conv7)
    pool_4_up = UpSampling2D(size=(2,2))(pool_4)
    
    merged = Concatenate([pool_3, pool_4_up, conv7_up])
    classify = BilinearUpSampling2D(size=(8, 8))(merged)
    
    model = Model(inputs, classify)
    
    return model
    

def FCN_VGG16(input_size, n_classes):
    
    vgg_model = vgg16.VGG16(input_shape=input_size, include_top=False)
    fc1 = Conv2D(4096, (7, 7), activation='relu', padding='same')(vgg_model.output)
    fc2 = Conv2D(4096, (1, 1), activation='relu', padding='same')(fc1)
    x = Conv2D(n_classes, (1,1), padding='valid', strides=(1,1))(fc2)
    classify = BilinearUpSampling2D(size=(32, 32))(x)
    
    model = Model(vgg_model.input, classify)
    
    return model

In [4]:
fcn_8 = FCN_8(input_size=(256, 256, 3), n_classes=2)
fcn_16 = FCN_16(input_size=(256, 256, 3), n_classes=2)
fcn_32 = FCN_32(input_size=(256, 256, 3), n_classes=2)
vgg_fcn = FCN_VGG16(input_size=(256, 256, 3), n_classes=2)

ValueError: Layer bilinear_up_sampling2d_1 was called with an input that isn't a symbolic tensor. Received type: <class 'keras.layers.merge.Concatenate'>. Full input: [<keras.layers.merge.Concatenate object at 0xb27672da0>]. All inputs to the layer should be tensors.

In [ ]:
fcn_32.summary()

In [ ]:
vgg_fcn.summary()